# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_clean=pd.read_csv("../output_data/cities_clean.csv")


In [3]:
cities_clean.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.60,73.08,84.85,70.0,79.0,8.43,MV,1593153683
1,Vaini,-21.20,-175.20,75.20,83.0,75.0,5.82,TO,1593153988
2,Araouane,18.90,-3.53,92.25,23.0,50.0,9.91,ML,1593154069
3,Provideniya,64.38,-173.30,44.60,65.0,75.0,15.66,RU,1593153983
4,Youghal,51.95,-7.85,60.01,87.0,75.0,4.70,IE,1593154069


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations=cities_clean[["Lat","Lng"]]

In [6]:
rating = cities_clean["Humidity"].astype(float)

max_humidity=cities_clean["Humidity"].max()

print(max_humidity)


100.0


In [7]:
# Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius=2,opacity=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
cities_weather_best=cities_clean[((cities_clean["Max Temp"]>70) & (cities_clean["Max Temp"]<80))&(cities_clean["Wind Speed"]<10)&(cities_clean["Cloudiness"]==0)]

In [9]:
cities_weather_best.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Noumea,-22.28,166.46,72.00,60.0,0.0,3.36,NC,1593154070
1,Morondava,-20.28,44.28,76.01,70.0,0.0,8.37,MG,1593154044
2,Kresttsy,58.25,32.52,78.06,51.0,0.0,4.27,RU,1593154072
3,Faya,18.39,42.45,78.80,17.0,0.0,9.17,SA,1593153983
4,Cabo San Lucas,22.89,-109.91,78.01,88.0,0.0,8.10,MX,1593153826
5,Yeppoon,-23.13,150.73,73.40,31.0,0.0,4.70,AU,1593153950
6,Kyotera,-0.62,31.52,77.41,46.0,0.0,8.79,UG,1593154075
7,Rakitnoye,50.84,35.85,77.00,50.0,0.0,8.95,RU,1593154075
8,Mbarara,-0.61,30.65,72.70,57.0,0.0,6.73,UG,1593154049
9,Abhā,18.22,42.51,78.80,17.0,0.0,9.17,SA,1593154077


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df=pd.DataFrame(cities_weather_best).reset_index(drop=True)

In [11]:
hotel_df["Name"]=""

hotel_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'Name'],
      dtype='object')

In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# run a request using our params dictionary
for index, row in hotel_df.iterrows():
#for i in range(len(hotel_df["City"])):
    #print(hotel_df[hotel_df["City"]==city]["Lat"])
    #print(hotel_df[hotel_df["City"]==city]["Lng"])
    #print(city)
    #hotel_df[hotel_df.loc[i,"City"]==city]["Lat"].astype(str)+", "+hotel_df[hotel_df.loc[i,"City"]==city]["Lng"].astype(str)
    params["location"]=str(row["Lat"])+", "+str(row["Lng"])
    
    response = requests.get(base_url, params=params)#.json()
    #print(response.url)
    hotel_data = response.json()
    
    #print(index)
    
    #print(hotel_data["results"][0]["name"])
    hotel_df.loc[index,"Name"]=hotel_data["results"][0]["name"]
    # Print the json (pretty printed)
    #print(json.dumps(places_data, indent=4, sort_keys=True))

In [13]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Name
0,Noumea,-22.28,166.46,72.00,60.0,0.0,3.36,NC,1593154070,Maison de quartier de la Vallée-des-Colons
1,Morondava,-20.28,44.28,76.01,70.0,0.0,8.37,MG,1593154044,Lycee Pole Morondava I
2,Kresttsy,58.25,32.52,78.06,51.0,0.0,4.27,RU,1593154072,Rostelecom
3,Faya,18.39,42.45,78.80,17.0,0.0,9.17,SA,1593153983,مقبرة ال مجمل
4,Cabo San Lucas,22.89,-109.91,78.01,88.0,0.0,8.10,MX,1593153826,Cervecería La Pintada
5,Yeppoon,-23.13,150.73,73.40,31.0,0.0,4.70,AU,1593153950,Vaughan's Furniture
6,Kyotera,-0.62,31.52,77.41,46.0,0.0,8.79,UG,1593154075,Nkenge play ground
7,Rakitnoye,50.84,35.85,77.00,50.0,0.0,8.95,RU,1593154075,"Aleksandrovskiy, Dk"
8,Mbarara,-0.61,30.65,72.70,57.0,0.0,6.73,UG,1593154049,RTO of Jehovah's Witneses
9,Abhā,18.22,42.51,78.80,17.0,0.0,9.17,SA,1593154077,المتوسطة السادسة للبنات


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))